## System Efficiency...the clock matters

### Introduction
In Module 7 we discussed NLP system performance in terms of the *effectiveness* of an NLP system compared to human annotations. We focused on measures like recall, precision, and accuracy. In this notebook we will explore another measure of NLP systems, system *efficiency*. Testing an NLP system's efficiency asks, "How fast is it?" Or, put another, more practical way, "Is the system *fast enough?*"

In the real world of clinical NLP, increasingly we want to process ever more notes. NLP technology, and the hardware that supports it, improved dramatically over the past 15 years. Critically, hardware costs have plummeted at the same time as the HITECH Act made vast quantities of clinical data easier to create and exchange.

But as we process more and more notes we need to be mindful of how long it takes to do so. Processing efficiency becomes increasingly important as the size of clinical datasets grows. In clinical natural language processing (NLP), even small increases in processing throughput are important when handling very large note corpora. 
### The Clock on the Wall
An extreme example is the U.S. Veterans Administration VINCI national data warehouse, which contains 2-3 billion notes. Divita et al. demonstrated the effect of note processing efficiency in VINCI studies (see the required article for this Module). In their work, 6 million records is considered a representative national sample for many applications. Shaving off 100 milliseconds (i.e., one-tenth of second) of processing time per note in their benchmark system (401 milliseconds was their nominal per-note-processing time) **would save nearly a week** of clock time for a corpus of that size.

To assess efficiency, we focus on “clock time,” the amount of real time, from the human perspective, a process takes to complete. To put clock time into a *real-world context, see the real-time, near real-time, and daily examples* in the Module 10 lecture. In this notebook, let’s explore the scenario (a very practical one as you start to build your project!) where you, as a researcher, are tweaking an NLP algorithm and have to wait for processing after each tweak.






### The `timeit` library
First we need some Python tools that can provide a stopwatch for measuring how long Python processes take to run. These will allow us to see where clock-time bottlenecks are. For more information on the timeit library, [look here in the Python documention.](https://docs.python.org/2/library/timeit.html#)



In [1]:
# As usual, let's import what we need...
import timeit
import numpy as np  


In [2]:
# Now we define a test function that we can use to practice:
def test():
    '''A test function that builds a list of the first 10,000 integers by appending them -- nothing fancy'''
    mylist = []
    for i in range(100000):
        mylist.append(i)
        

`Timeit` provides a new class and a few methods to measure the time it takes to execute statements. To keep things simple, we won't worry about the class business now, just focus on the methods. One useful method is: 

`timeit.repeat(stmt = '<the Python code statement you intend to time>', \
    setup= '<any Python code to run each time the REPEAT below starts>' \  #optional
    repeat = <number>, \ #i.e., how many times to REPEAT the number of runs below AND average the results \
    number = <number>) #how many times to run the stmt code, i.e.,the TOTAL runs is (repeat*number)`

In the first example, we will run test(), which appends 10,000 times, for 10 batches (i.e., `number = 10`)

In [3]:
#1) Run test() in loops of 10, average those, and repeat 3 times (the default if repeat is not specified):
runtime = timeit.repeat(stmt = 'test()', 
                        setup = 'print(".", end = ""); from __main__ import test', #print a period after each repeat
                        number = 10)
print(' The default repeat is 3, so 30 runs of test(), or **300,000** appends total\n for a series of ', \
      runtime, ' seconds.')

... The default repeat is 3, so 30 runs of test(), or **300,000** appends total
 for a series of  [0.15647347690537572, 0.09506166819483042, 0.09126702509820461]  seconds.


Pretty fast. 

How may 100ths of seconds, on average, did the three runs take to append 10,000 times in`test()`?

In [4]:
print(np.mean(runtime),"seconds.")

0.114267390066 seconds.


In [5]:
#2) Now run test() in batches of 100 and repeat 10 times, then average the time for each repeat loop
runtime = (timeit.repeat(stmt = 'test()', 
                         setup = 'print(".", end = ""); from __main__ import test', #print a period after each repeat
                         repeat = 10, 
                         number = 100))
print('Repeat 10 batches of 100 runs and average the times for batch, **10,000,000** appends total\n for a series of ', runtime, ' seconds.')


..........Repeat 10 batches of 100 runs and average the times for batch, **10,000,000** appends total
 for a series of  [1.013889370020479, 0.9408897641114891, 0.9301630049012601, 0.9401369169354439, 0.9117680923081934, 0.9292277037166059, 0.9321360560134053, 0.9311231565661728, 0.9579355088062584, 0.929843776859343]  seconds.


In [6]:
print(np.mean(runtime),"seconds.")

0.941711335024 seconds.


Again, pretty fast. Compare the overall average for both runs above...is it what you expected? 

Run the two cells above again. You will see that the mean time is slightly different. **Why is that?** Because the CPU is multitasking. If this Python notebook was the only process running on the CPU, the times would be much more consistent. But you are sharing the CPU with the rest of the class and with everyone else using the server. And, of course, the server has several tasks of its own to do. **That is why, when timing code, it's important to average over several runs to smooth out the interference from other processes**.

In [7]:
# 3) Okay, now let's increase the number of runs one more time: Run test() in batches of 1,000, average \
# the results, and repeat 5 times:
print("--> WAIT for it!")
runtime = timeit.repeat(stmt = 'test()', 
                    setup = 'print(".", end = ""); from __main__ import test', #print a period after each repeat
                    repeat = 3, 
                    number = 1000)
print("\nThe average time to append 10,000 integers 1,000 times (from 3 repeats) is ", np.mean(runtime), "seconds")

--> WAIT for it!
...
The average time to append 10,000 integers 1,000 times (from 3 repeats) is  9.34650781921 seconds


Again, do these execution times for the three examples seem about what you expected? **It is reassuringly linear**, that is, every time we increase the number of runs by a factor of 10 the execution time also increases roughly by a factor of 10.

#### Now lets time some NLP processes...
One annoying feature of `timeit` is that it really only wants to clock a single statement. That means that you need to put the code that you want to time inside of a function. It also means you need to `import` that function's parameters, as shown below. *If you find a way to time multiple statements in timeit, do let the class know! The documentation implies it is possible, but it is not clear to us how....* 

We are going to pinch code from the implementation notenook back in Module 7, the code that segments a note and then runs the PyConText assertion process on those segments. **Note:** in your project, and in the real world, you would be processing hundreds or thousands of notes in a typical pipeline run. We *simulate* that in this exercise by timing the pipeline as it runs on one note hundred of times, over and over. This is not a brilliant approach to simulation, because any given note might be particularly easy or particularly hard for segmentation and context assertion. 

**At the end of the notebook, if you are you feeling adventurous, pinch the code from an earlier notebook, read in a hundred notes, and loop over them with the timer. :) If you do that, you will notice a decidedly different mean runtime per note.**


In [10]:
#Do the usual imports
from pipeUtils import Document
from pipeUtils import Annotation

from wrapperPyConText import ConTextPipe
from PyRuSH.RuSH import RuSH

#Now create a document instance and populate it with grab a single note and its annotations
doc = Document() #create a document instance
doc.load_document_from_file('data/test.txt') #use the document methods to read in the note and its annotations
doc.load_annotations_from_brat('data/test.ann')

#Next, read in the sementer rule set, in the KB directory, and create a sentence segmenter instance
sentence_rules= 'KB/rush_rules.tsv'
sentence_segmenter = RuSH(sentence_rules)

The information extraction rule set below is teeny. This is really a toy set of rules; a real pipeline will have far more rules and thus will run more slowly than our experiments here.

In [11]:
#Load the information extraction rules. This is a toy set, a real pipeline will have many ruch rules.
#and thus a real pipeline will be slower.
target_rules='''
Comments: ''
Direction: ''
Lex: pain
Regex: ''
Type: Depression_Symptom
---
Comments: ''
Direction: ''
Lex: depression
Regex: 'depres\\w+'
Type: Depression_Symptom
---
Comments: ''
Direction: ''
Lex: suicidal
Regex: 'suicidal\\s*ideation'
Type: Depression_Symptom
'''

To use `timeit`, we need to wrap the segmenter and context code into a function we can time.

In [12]:
def do_anns(document):
    '''A test function that wrap a document segmenter and context asserter so we can time the runtime of both.'''
    #segment the document's sentences
    sentences=sentence_segmenter.segToSentenceSpans(doc.text)
    i=0
    for sentence in sentences: 
        i=i+1  #incrementing the index
        doc.annotations.append(Annotation(start_index=sentence.begin,
                                          end_index=sentence.end, 
                                          type="Sentence", 
                                          ann_id='Sent_'+str(i)))
    #now run context on each sentence segment
    context_pipe = ConTextPipe(target_rules, rule_set)
    doc.annotations = [] #a subtle 'persistence' side-effect here; if not reset it accumulates across runs
    for a in doc.annotations: #loop through the sentences and run the context algorithm over each
        if a.type == "Sentence":
            b = context_pipe.process(a, doc.text)
            doc.annotations.extend(b)
                

Now we are all set to time the new "segmenter-context" function. We will make three runs, each time adding more context rules. More rules mean more *effective* context assertion, but that comes at the price of lower efficiencey (slower performance), as we will see.

In [13]:
rule_sets = ['KB/general_modifiers_50-rules.yml', \
             'KB/general_modifiers_half-rules.yml', \
             'KB/general_modifiers_all-rules.yml']

for rule_set in rule_sets:
    runtime = timeit.repeat('do_anns(doc)',
                            setup = 'print(".", end = ""); from __main__ import do_anns, doc', #ugh...note the import
                            repeat = 3, 
                            number = 10) ##simulates a 10 note run
    print("Time(s) for rule set ",rule_set,"for 10 notes: ", runtime)
              

...Time(s) for rule set  KB/general_modifiers_50-rules.yml for 10 notes:  [1.6463442277163267, 1.5755672696977854, 1.591118745971471]
...Time(s) for rule set  KB/general_modifiers_half-rules.yml for 10 notes:  [2.702311024069786, 2.6901290197856724, 2.7027526618912816]
...Time(s) for rule set  KB/general_modifiers_all-rules.yml for 10 notes:  [4.270881077740341, 4.2291063903830945, 4.3078813180327415]


In [30]:
%%timeit -r 10 -n 1 -p 5 #-s 'print(".", end = "")'
rule_sets = ['KB/general_modifiers_all-rules.yml']

for rule_set in rule_sets:
    sentences=sentence_segmenter.segToSentenceSpans(doc.text)
    i=0
    for sentence in sentences: 
        i=i+1  #incrementing the index
        doc.annotations.append(Annotation(start_index=sentence.begin,
                                          end_index=sentence.end, 
                                          type="Sentence", 
                                          ann_id='Sent_'+str(i)))
    #now run context on each sentence segment
    context_pipe = ConTextPipe(target_rules, rule_set)
    doc.annotations = [] #a subtle 'persistence' side-effect here; if not reset it accumulates across runs
    for a in doc.annotations: #loop through the sentences and run the context algorithm over each
        if a.type == "Sentence":
            b = context_pipe.process(a, doc.text)
            doc.annotations.extend(b)

UsageError: option -s not recognized ( allowed: "n:r:tcp:qo" )


##### The "half-rules.yml" context rule set has about 190 rules, while the "all-rules.yml" set has about 380 rules. Yet these runs times are **not so linear**. Why is that, do you think?

#### OPTIONAL: Try reading a hundred notes from MIMIC II and timing the pipeline here on each note, rather than simply procesing the same note repeatedly like we do above. To ge you started, here's the SQL:


In [8]:
import pymysql
import pandas as pd
import getpass
import re

conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')

Enter MySQL passwd for jovyan········


In [9]:
notes = pd.read_sql("SELECT DISTINCT * FROM noteevents limit 100",conn)

### WRITE YOUR timer here

In [ ]:
%%timeit -r 1000 -n 10 -p 5












In [14]:
#Here's ours:
print(notes.text[0:6])
rule_set = 'KB/general_modifiers_all-rules.yml'
for index, note in notes.iterrows():
    new_note = note.text.replace('\n',' ')
    doc = Document()
    doc.text = new_note  #sets up the text for the next note to run throgh the pipe
    doc.load_annotations_from_brat('data/test.ann')#since we don't have annotations yet, just use the test annotations
    if re.match(r'(\s*)(\w+)', doc.text) != None:   #make sure the note has characters in it, else RUSH could crash
        runtime = timeit.repeat('do_anns(doc)',
                                setup = 'from __main__ import do_anns, doc', #ugh...note the import
                                repeat = 1, 
                                number = 1)
        print("Time(s) for note ",index,": ", runtime)
    else:
        print("Empty note...",index)

0    \n \n \n \nAdmission Date:  [**2644-1-17**]   ...
1    \n\n\n     DATE: [**2644-1-17**] 10:53 AM\n   ...
2    \n\n\n     DATE: [**2644-1-17**] 10:43 AM\n   ...
3    \n\n\n     DATE: [**2644-1-17**] 6:37 AM\n    ...
4    \nNSG Admit noteB:\nPlease refer to careview a...
5    \nNursing Progress Note:\nPlease refer to Care...
Name: text, dtype: object
Time(s) for note  0 :  [0.8701527533121407]
Time(s) for note  1 :  [0.47419741470366716]
Time(s) for note  2 :  [0.4613146777264774]
Time(s) for note  3 :  [0.4774014907889068]
Time(s) for note  4 :  [0.41591173131018877]
Time(s) for note  5 :  [0.4160619741305709]
Time(s) for note  6 :  [0.42232962930575013]
Time(s) for note  7 :  [0.3776710261590779]
Time(s) for note  8 :  [0.47029974590986967]
Time(s) for note  9 :  [1.2523656389676034]
Time(s) for note  10 :  [0.4285039622336626]
Time(s) for note  11 :  [0.41606609942391515]
Time(s) for note  12 :  [0.45940754003822803]
Time(s) for note  13 :  [0.48897488275542855]
Time(s) for note 

### Try plotting these times as a histogram (hint, save the runtimes right in the notes dataframe...)